In [1]:
import numpy as np

class Global(object):
    """このモジュールは設計変数を持った個体を目的関数で評価した状態にして制約条件を考慮して
        正規化し、初期化と適者生存を行う
        func():評価関数、目的関数を出す部分
        individual():個体を設定する部分
        initialize():初期個体を形成する部分
        variation():自然交配を行う部分"""
    
    def __init__(self,individual_num,design_num,objective_num):
        """individual_num:設計で用いる個体数
           design_num:設計変数の個数
           objective_num:目的関数の数"""
        
        self.d=design_num
        self.N=individual_num
        self.M=objective_num
        
        #正規化した時に範囲は[0,1]
        self.lower=-np.ones((1,design_num))
        self.upper=np.ones((1,design_num))
        
    """評価関数の部分"""    
    def func(self,x):
        """calculate the objective func vector
           x: decision vectors
           return: objective vectors"""
        n=x.shape[0]
        a=np.zeros((self.M,self.d))
        
        for i in range(self.d):
            for j in range(self.M):
                a[j,i]=((i+0.5)**(j-0.5))/(i+j+1.0)
        obj=np.zeros((n,self.M))
        
        for i in range(n):
            for j in range(self.M):
                obj[i,j]=np.dot(x[i,:]**(j+1),a[j,:].T)
                
        return obj
    
    def individual(self,decs):
        """turn decision vectors into individual
           return: pop_obj,decs
           目的関数で評価された個体と決定ベクトル"""
        pop_obj=self.func(decs)
        
        return [pop_obj,decs]
    
    """初期化と正規化"""
    def initialize(self):
        pop_decs=np.random.random((self.N,self.d))*(self.upper-self.lower)+self.lower
        
        return self.individual(pop_decs)
    
    """適者生存選択　交叉"""
    def variation(self,pop_dec,boundary=None):
        """return: offspring_decs"""
        pro_c=1
        dis_c=20
        pro_m=1
        dis_m=20
        pop_dec=pop_dec[:len(pop_dec)//2*2][:]#pop_dec=np.shape((N,d))
        (n,d)=np.shape(pop_dec)
        parent_1_dec=pop_dec[:n//2,:]
        parent_2_dec=pop_dec[n//2:,:]
        beta=np.zeros((n//2,d))
        mu=np.random.random((n//2,d))
        beta[mu<=0.5]=np.power(2*mu[mu<=0.5],1/(dis_c+1))
        beta[mu>0.5]=np.power(2*mu[mu>0.5],-1/(dis_c+1))
        beta=beta*((-1)**np.random.randint(2,size=(n//2,d)))
        beta[np.random.random((n//2,d))<0.5]=1.0
        beta[np.tile(np.random.random((n//2,1))>pro_c,(1,d))]=1.0
        #子供個体
        offspring_dec=np.vstack(((parent_1_dec+parent_2_dec)/2+beta*(parent_1_dec-parent_2_dec)/2,
                               (parent_1_dec+parent_2_dec)/2-beta*(parent_1_dec-parent_2_dec)/2))
        
        site=np.random.random((n,d))< pro_m/d
        mu=np.random.random((n,d))
        
        """mu<=0.5の処理"""
        temp=site&(mu<=0.5)#mu<0.5を満たしているかをbool値で返す
        
        if boundary is None:
            lower,upper=np.tile(self.lower,(n,1)),np.tile(self.upper,(n,1))
        else:
            lower,upper=np.tile(boundary[0],(n,1)),np.tile(boundary[1],(n,1))
            
        norm=(offspring_dec[temp]-lower[temp])/(upper[temp]-lower[temp])
        offspring_dec[temp]+=(upper[temp]-lower[temp])*(np.power(2*mu[temp]+(1.0-2.0*mu[temp])*np.power(1.0-norm,dis_m+1.0),1.0/(dis_m+1.0)))
        
        
        #mu>0.5
        temp=site&(mu>0.5)
        norm=(upper[temp]-offspring_dec[temp])/(upper[temp]-lower[temp])
        offspring_dec[temp]+=(upper[temp]-lower[temp])*(1.0-np.power(2.0*(1.0-mu[temp])+2.0*(mu[temp]-0.5)*np.power(1.0-norm,dis_m+1),1.0/(dis_m+1.0)))

        offspring_dec=np.maximum(np.minimum(offspring_dec,upper),lower)#閾値をカット
        
        return offspring_dec
        
        
    

In [2]:
#非優劣ソート
import numpy as np

def nd_sort(pop_obj,n_sort):
    """pop_obj:目的関数で評価された個体
       n_sort:ランクソートする数"""
    n,m_obj=np.shape(pop_obj)
    a,loc=np.unique(pop_obj[:,0],return_inverse=True)#loc:重複なしのインデックス
    index=pop_obj[:,0].argsort()#ソートしてインデクス抽出
    new_obj=pop_obj[index,:]#新しい個体の初期化
    front_no=np.inf*np.ones(n)#pareto解のリスト
    max_front=0#rank
    
    while np.sum(front_no<np.inf)<min(n_sort,len(loc)):
        max_front+=1
        for i in range(n):
            if front_no[i]==np.inf:
                dominated=False
                
                for j in range(i,0,-1):
                    if front_no[j-1]==max_front:
                        m=2
                        while (m<=m_obj) and (new_obj[i,m-1] >= new_obj[j-1,m-1]):
                            m+=1
                        dominated=m>m_obj#bool
                        
                        if dominated or (m_obj==2):
                            break
                if not dominated:
                    front_no[i]=max_front
                    
    return front_no[loc],max_front

In [3]:
#混雑度を計算
import numpy as np
def crowding_distance(pop_obj,front_no):
    """pop_obj:目的関数で評価された個体
       front_no:pareto解のリスト"""
    n,M=np.shape(pop_obj)
    crowd_dis=np.zeros(n)
    front=np.unique(front_no)
    Fronts=front[front!=np.inf]#front
    
    
    for f in range(len(Fronts)):
        Front=np.array([k for k in range(len(front_no)) if front_no[k]==Fronts[f]])#Frontのインデクス
        Fmax=pop_obj[Front,:].max(0)
        Fmin=pop_obj[Front,:].min(0)
        
        for i in range(M):
            rank=np.argsort(pop_obj[Front,i])#rankのリスト
            
            crowd_dis[Front[rank[0]]]=np.inf
            crowd_dis[Front[rank[-1]]]=np.inf
            
            for j in range(1,len(Front)-1):
                crowd_dis[Front[rank[j]]]=crowd_dis[Front[rank[j]]]+(pop_obj[(Front[rank[j+1]],i)]-pop_obj[(Front[rank[j-1]],i)])/(Fmax[i]-Fmin[i])
                
        
    return crowd_dis
            
    

In [6]:
import numpy as np

def tournament(K,N,fit):
    """params
       K:比較する個数
       N:選ぶ個数
       fit:fitness vector
       return: 選択したもののインデクス"""
    
    n=len(fit)
    mate=[]
    
    for i in range(N):
        a=np.random.randint(n)
        for j in range(K):
            b=np.random.randint(n)
            for r in range(len(fit[0])):
                if fit[(b,r)]<fit[(a,r)]:
                    a=b
                    
        mate.append(a)
    return np.array(mate)

In [7]:
#selection

def environment_selection(population,N):
    """population:現在の集団 [decision vectors,pop_obj]
       N:選択する個体の数
       return:next_pop,front_no,crowd_dis,index"""
    
    front_no,max_front=nd_sort(population[1],N)
    next_label=[False for _ in range(front_no.size)]#next_labelの初期化
    
    #個体数を超えないランクまで
    for i in range(front_no.size):
        if front_no[i]<max_front:
            next_label[i]=True
            
    crowd_dis=crowding_distance(population[1],front_no)#混雑度距離
    
    #残った中で最もランクが高いもの
    last=[i for i in range(len(front_no)) if front_no[i]==max_front]
    rank=np.argsort(-crowd_dis[last])
    delta_n=rank[:(N-int(np.sum(next_label)))]#Trueの個数を数える
    res=[last[i] for i in delta_n]
    
    for i in res:
        next_label[i]=True
        
    index=np.array([i for i in range(len(next_label)) if next_label[i]])
    next_pop=[population[0][index,:],population[1][index,:]]
    
    return next_pop,front_no[index],crowd_dis[index],index
    

In [8]:
#main nsga2

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

individual_num=100
design_num=10
objective_num=3

Global=Global(individual_num,design_num,objective_num)
class nsga2(object):
    
    def __init__(self,decs=None,ite=100,eva=100*500):
        self.decs=decs
        self.ite=ite
        self.eva=eva
        
    def run(self):
        """return:次世代の個体"""
        #個体の初期化
        if self.decs is None:
            population=Global.initialize()
        else:
            population=Global.individual(self.decs)
            
        front_no,max_front=nd_sort(population[1],np.inf)#pareto解群,rank
        crowd_dis=crowding_distance(population[1],front_no)#混雑度距離
        evaluation=self.eva
        
        while self.eva>=0:
            fit=np.vstack((front_no,crowd_dis)).T
            mating_pool=tournament(2,Global.N,fit)
            pop_dec,pop_obj=population[0],population[1]
            parent=[pop_dec[mating_pool,:],pop_obj[mating_pool,:]]#親個体
            offspring=Global.variation(parent[0],boundary=(Global.lower,Global.upper))#子供
            population=[np.vstack((population[0],Global.individual(offspring)[0])),np.vstack((population[1],Global.individual(offspring)[1]))]
            #自然交配
            population,front_no,crowd_dis,_=environment_selection(population,Global.N)
            
            self.eva-=Global.N
            
        return population
    
    def draw(self):
        """結果の描画"""
        population=self.run()
        pop_obj=population[1]
        front_no,max_front=nd_sort(pop_obj,1)#n_sort=1
        non_dominated=pop_obj[front_no==1,:]
        
        if Global.M==2:
            plt.scatter(non_dominated[0,:],non_dominated[1,:])
            
        elif Global.M==3:
            x,y,z=non_dominated[:,0],non_dominated[:,1],non_dominated[:,2]
            ax=plt.subplot(111,projection='3d')
            ax.scatter(x,y,z)
            
        else:
            for i in range(len(non_dominated)):
                plt.plot(range(1,Global.M+1),non_dominated[i,:])
        
            
        
        

In [9]:
nsga=nsga2()
nsga.draw()
plt.show()

IndexError: boolean index did not match indexed array along dimension 1; dimension is 10 but corresponding boolean dimension is 3